In [37]:
import pandas as pd
import numpy as np
import acquire
import wrangle_zillow
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

print('Done!')

Done!


In [28]:
df=acquire.load_zillow_data()
df.head()

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,buildingclasstypeid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,parcelid.1,logerror,transactiondate.1,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,buildingclassdesc
0,NaN,NaN,NaN,261.0,NaN,NaN,NaN,14297519,1727539,NaN,...,14297519,0.025595,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
1,NaN,NaN,NaN,261.0,NaN,NaN,NaN,17052889,1387261,NaN,...,17052889,0.055619,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
2,NaN,NaN,NaN,261.0,NaN,NaN,NaN,14186244,11677,NaN,...,14186244,0.005383,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
3,NaN,NaN,NaN,261.0,2.0,NaN,NaN,12177905,2288172,NaN,...,12177905,-0.103410,2017-01-01,NaN,NaN,Central,Single Family Residential,NaN,NaN,NaN
4,NaN,NaN,NaN,266.0,2.0,NaN,1.0,10887214,1970746,NaN,...,10887214,0.006940,2017-01-01,Central,NaN,Central,Condominium,NaN,NaN,NaN


In [ ]:
def columns_data(df):
    columns_data = pd.concat([pd.Series(df.isnull().sum()), pd.Series(df.isnull().sum()/df.shape[0])], axis=1)
    columns_data.reset_index(inplace = True)
    columns_data.columns=(['columns', 'num_rows_missing', 'pct_rows_missing'])
    print(f"The max pct_rows_missing - {columns_data['pct_rows_missing'].max()}")
    print(f"The average pct_rows_missing - {columns_data['pct_rows_missing'].mean()}")
    print(f"The median pct_rows_missing - {columns_data['pct_rows_missing'].median()}")
    return columns_data

column_data = columns_data(df)

In [ ]:
column_data

In [ ]:
column_data['keep'] = column_data['pct_rows_missing'] <= .3
column_data['columns'][column_data['keep'] == True]

In [ ]:
['propertylandusetypeid', 'parcelid', 'id', 'bathroomcnt', 'bedroomcnt',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertycountylandusecode',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'transactiondate', 'parcelid.1',
       'logerror', 'transactiondate.1', 'propertylandusedesc']
      dtype='object'

In [ ]:
counts = pd.DataFrame(df.isnull().sum(axis=1).value_counts().sort_values())
counts.reset_index(inplace = True)
counts.columns = ['num_cols_missing', 'num_rows']
counts['pct_cols_missing'] = counts['num_cols_missing']/df.shape[1]
counts = counts.sort_values('num_cols_missing').reset_index(drop= True)
print(f"The average num_rows missing - {counts['num_rows'].mean()}")
counts

In [ ]:
def summarize(df):
    print(f"INFO \n {df.info} \n")
    print(f"DESCRIPTION\n {df.describe()} \n")
    print(f"SHAPE \n {df.shape} \n")
    
summarize(df)

In [34]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    This function takes in a DataFrame (df), a minium for prop_required_column [0:1], 
    and a minimum value [0:1] for prop_required_row.
    
    It will first drop columns who's missing data is less than prop_required_column value.
    It will then drop the rows who's missing data is lower than the prop_required_row.
    '''
    columns_to_keep = []
    columns = df.columns
    for col in columns:   
        if df[col].notnull().sum()/df.shape[0] >= prop_required_column:
            columns_to_keep.append(col)
    df = df[columns_to_keep]
    df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row
    df = df.loc[df['row_to_keep'] == True]
    df.drop(columns= ['row_to_keep'], inplace = True)
    return df

In [ ]:
df1 = handle_missing_values(df, .6, .75)
df1.shape

In [33]:
def handle_missing_values2(df, prop_required_column = .7, prop_required_row = .6):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

df2 = handle_missing_values2(df, .5, .1)
print(df2.shape)
df2.columns

(43779, 36)


Index(['propertylandusetypeid', 'heatingorsystemtypeid', 'parcelid', 'id',
       'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock',
       'transactiondate', 'parcelid.1', 'logerror', 'transactiondate.1',
       'heatingorsystemdesc', 'propertylandusedesc'],
      dtype='object')

In [ ]:
['propertylandusetypeid', 'parcelid', 'id', 'bathroomcnt', 'bedroomcnt',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertycountylandusecode',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'transactiondate', 'parcelid.1',
       'logerror', 'transactiondate.1', 'propertylandusedesc']

In [ ]:
df = df[df.propertylandusetypeid.isin([261, 262, 273])]
df.drop(columns = ['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
       'propertylandusetypeid', 'heatingorsystemtypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id', 'unitcnt'], inplace = True)
df.drop(columns = [c for c in df.columns if c.endswith('.1')], inplace = True)
df['has_basement'] = df['basementsqft'] > 0
df.replace({True:1, False:0}, inplace = True)
df['fireplaceflag']
df.columns

In [ ]:
df[['airconditioningtypeid']].isnull().sum()

In [ ]:
df['fireplacecnt'].isnull().sum()

In [ ]:
df = wrangle_zillow.prep_zillow()
df.shape

In [ ]:
df.isnull().mean().sort_values(ascending=False)

In [ ]:
df['regionidcity'].mode()

In [ ]:
from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df, test_size=.2, random_state=333)
train, validate = train_test_split(train_validate, test_size=.25, random_state=333)

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
train['propertyzoningdesc'].value_counts()

In [2]:
train, validate, test = wrangle_zillow.prep_zillow()
train.isnull().sum().sort_values(ascending=False)

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\anthony\Documents\Data_Science\CodeUp\clustering-exercises\wrangle_zillow.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row
C:\Users\anthony\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

has_basement                    0
propertylandusedesc             0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertyzoningdesc              0
rawcensustractandblock          0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
transactiondate                 0
logerror                        0
heatingorsystemdesc             0
bathroomcnt                     0
dtype: int64

In [ ]:
# Categorical/Discrete columns to use mode to replace nulls

med_cols = [
    "taxamount",
    "calculatedfinishedsquarefeet",
    "structuretaxvaluedollarcnt",
    "lotsizesquarefeet"
    ]

for col in med_cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)
    
mod_cols = [
    "yearbuilt",
    "regionidzip",
    "propertyzoningdesc",
    "buildingqualitytypeid",
]

for col in mod_cols:
    mode = str(train[col].mode())# I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [ ]:
mod_cols = [
    "yearbuilt",
    "regionidzip",
    "propertyzoningdesc",
    "buildingqualitytypeid",
]

for col in mod_cols:
    mode = str(train[col].mode())# I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
train.dtypes

# Reassess

In [51]:
df = acquire.load_zillow_data()
df = df[df.propertylandusetypeid.isin([261, 262, 273])]
df.drop(columns = ['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
    'propertylandusetypeid', 'heatingorsystemtypeid',
    'architecturalstyletypeid', 'airconditioningtypeid', 'id', 'parcelid'], inplace = True)
df.drop(columns = [c for c in df.columns if c.endswith('.1')], inplace = True)
df = df[df.unitcnt == 1.0]
df['has_basement'] = df['basementsqft'] > 0
df.replace({True:1, False:0}, inplace = True)
df['heatingorsystemdesc'].fillna('None', inplace=True)
df = handle_missing_values(df, .6, .6)
df.drop(columns=['calculatedbathnbr', 'unitcnt', 'fullbathcnt', 'propertyzoningdesc'], inplace = True)

train_validate, test = train_test_split(df, test_size=.2, random_state=369)
train, validate = train_test_split(train_validate, test_size=.25, random_state=369)

In [52]:
print(f" Number of rows train, validate, test {train.shape[0], validate.shape[0], test.shape[0]}")
train.isnull().sum().sort_values(ascending=False)

 Number of rows train, validate, test (20240, 6747, 6747)


regionidcity                    420
lotsizesquarefeet               192
buildingqualitytypeid            88
censustractandblock              39
structuretaxvaluedollarcnt       39
regionidzip                       8
yearbuilt                         6
taxamount                         4
rawcensustractandblock            0
bedroomcnt                        0
calculatedfinishedsquarefeet      0
finishedsquarefeet12              0
fips                              0
latitude                          0
longitude                         0
propertycountylandusecode         0
has_basement                      0
regionidcounty                    0
propertylandusedesc               0
roomcnt                           0
taxvaluedollarcnt                 0
assessmentyear                    0
landtaxvaluedollarcnt             0
transactiondate                   0
logerror                          0
heatingorsystemdesc               0
bathroomcnt                       0
dtype: int64

In [53]:
train.fips.value_counts()

6037.0    20238
6059.0        2
Name: fips, dtype: int64

In [54]:
med_cols = [
"taxamount",
"calculatedfinishedsquarefeet",
"structuretaxvaluedollarcnt",
"lotsizesquarefeet"
]

for col in med_cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

mod_cols = [
    "yearbuilt",
    "regionidzip",
    "buildingqualitytypeid",
    'regionidcity',
    'censustractandblock'
]

for col in mod_cols:
    mode = str(train[col].mode())# I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [55]:
train.isnull().sum()

bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
transactiondate                 0
logerror                        0
heatingorsystemdesc             0
propertylandusedesc             0
has_basement                    0
dtype: int64